In [41]:
from random import randint
from copy import deepcopy

infile = open('input1.txt', 'r')

#Create adjacency list
def CreateAdjacencyList(infile):
    
    adj_list = {}
    circuit_max = 0
    for line in infile:
        node = line.strip('\n')
        node = node.replace(' -> ', ' ')
        node = node.split(' ')
        adj_list.setdefault(node[0], [])
        for number in node[1].split(','):
            adj_list[node[0]].append(number)
            circuit_max += 1
    
    return adj_list, circuit_max

#Find start/end nodes
def FindStart(red_adj_list):
    
    start = {}
    for one in red_adj_list:
        start.setdefault(one, 0)
        start[one] += len(red_adj_list[one])

    end = {}
    for one in red_adj_list:
        for two in red_adj_list[one]:
            end.setdefault(two, 0)
            end[two] += 1

    for one in end:
        try: 
            if start[one] != end[one]:
                if start[one] > end[one]:
                    start_node = one
                if start[one] < end[one]:
                    end_node = one
        except KeyError:
            end_node = one

    for one in start:
        try:
            if end[one] != start[one]:
                if end[one] < start[one]:
                    start_node = one
                if end[one] > start[one]:
                    end_node = one
        except KeyError:
            start_node = one

    red_adj_list[end_node] = []
    
    return red_adj_list, start_node

def FindEulerianPath(infile):
    
    #Create adjacency list
    adj_list, circuit_max = CreateAdjacencyList(infile)
    
    #Reduced adjacency list to keep track of traveled edges  
    red_adj_list = {}            
    red_adj_list = deepcopy(adj_list)
    
    #Find start node (graph must be directed/ubalanced)
    red_adj_list, start_node= FindStart(red_adj_list)
    
    start = start_node
    curr_vrtx = start_node
    path = [curr_vrtx]
    stack = []
    circuit = []
    while len(circuit) != circuit_max:
    
        if red_adj_list[curr_vrtx] != []: #If neighbors exist
            stack.append(curr_vrtx)
            pick = randint(0,len(red_adj_list[curr_vrtx])-1)
            temp = deepcopy(curr_vrtx)
            curr_vrtx = red_adj_list[temp][pick]
            red_adj_list[temp].remove(curr_vrtx)
        
        else:
            circuit.append(curr_vrtx)
            curr_vrtx = stack[len(stack)-1]
            stack.pop()
    
    #Formatting
    path = start + '->'
    for vrtx in circuit[::-1]:
        path += (vrtx + '->')
    return path.strip('->')
with open("output.txt", "w") as fd:
    fd.write( FindEulerianPath(infile))

In [44]:
import sys
import timeit
import random
from itertools import combinations,product,chain
from collections import Counter,defaultdict

def read_file(input_file):
    '''
    >>> k = read_file('test.k-universal_circular_string.4.txt')
    '''
    f = open(input_file)
    data = f.read().strip()
    f.close()
    return int(data)

def generate_binary(k):
    return [bin(i)[2:].zfill(k) for i in range(2**k)]

def form_relation(k):
    nodes = generate_binary(k-1)
    d = {}
    for item in nodes:
        add1 = item[1:]+'0'
        add2 = item[1:]+'1'
        d[item] = [add1,add2]
    return d

def form_cycle(data):
    rgk = random.choice(list(data.keys()))
    rgv = random.choice(data[rgk])
    cycle = [rgk]
    while len(data) != 0:
        try:
            cycle.append(rgv)
            if len(data[rgk]) > 1:
                data[rgk].remove(rgv)
            else:
                del data[rgk]
            rgk = rgv
            rgv = random.choice(data[rgk])
            if rgv == cycle[0] and rgv == cycle[-1]:
                break
        except:
            break
    return (cycle,data)

def form_unCycle(data,rgk):
    choose = data[rgk]
    rgv = random.choice(choose)
    cycle = [rgk]
    while len(data) != 0:
        try:
            cycle.append(rgv)
            if len(data[rgk]) > 1:
                data[rgk].remove(rgv)
            else:
                del data[rgk]
            rgk = rgv
            rgv = random.choice(data[rgk])
            if rgv == cycle[0] and rgv == cycle[-1]:
                break
        except:
            break
    return (cycle,data)

def fuse(Cycle,new_Cycle):
    fuse_index = Cycle.index(new_Cycle[0])
    return Cycle[:fuse_index]+new_Cycle+Cycle[fuse_index+1:]

def eulerian_cycle(data,k):
    Cycle,unCycle = form_cycle(data)
    while len(unCycle) != 0:
        keys = unCycle.keys()
        potential = list(set(Cycle)&set(keys))
        newStart = random.choice(potential)
        new_Cycle,unCycle = form_unCycle(unCycle,newStart)
        Cycle = fuse(Cycle,new_Cycle)
    return Cycle

def form_string(path):
    string = ''.join([item[-1] for item in path[1:]])
    return string

def result():
    k = 9
    data = form_relation(k)
    path = eulerian_cycle(data,k)
    results = form_string(path)
    print(results)
    return results
result()


11100100011000011001111100111101000000011000101111111110100110100001101111011001110010011001100100100111111000111100101001001011100101111010100110001110111100010010100001000111001100000111000001010111010010000011010110010110101011000000000101101111101101001011000110010101001010110110101111100001011101101100100001010001001101110101110000111101111110111011100111011000100010101011110011010100000100001110101010100011111010110100010110011011000010011100011010011101000110110111000101001111000000100100010000001111


'11100100011000011001111100111101000000011000101111111110100110100001101111011001110010011001100100100111111000111100101001001011100101111010100110001110111100010010100001000111001100000111000001010111010010000011010110010110101011000000000101101111101101001011000110010101001010110110101111100001011101101100100001010001001101110101110000111101111110111011100111011000100010101011110011010100000100001110101010100011111010110100010110011011000010011100011010011101000110110111000101001111000000100100010000001111'

Notes: We want to incorporate sequencing errors in the fragments f1 and f2 and get the confidence whether f1 and f2 belong from the same haplotype. Use some probablity to guess whether f1 and f2 belong to the same haplotype. 

Now once we have a probablity that f1 and f2 belong to the same haplotype, we want to check whether a variant is a false variant.

What is a false variant ? 
False variant is a location where the alleles are homozygous but the sequencing 
assumes / wrongly interprets the site as heterozygous. 
Hitherto, in the fragments (for these false variants) there would be a substantial amount of fragments where this site has the same allele say 0 for 
both fragments coming from both H1 and H2 (as it is not really a variant hence same allele is observed irrespective sample from H1 or H2 (0, 0) in the given position).

However for sequencing errors the errors are randomly distributed among different variants for different fragments.

Problem statement, we want to identify these false variant locations for the 
sequenced data, as these become difficult to handle while performing Hap assembly.

When we have a good coverage for a particular false variant, 
if we have fragment f1 which belongs to H1 and a fragment f2 which belongs to H2, the value for the variants are complement of each other except for the false variant. 

Now we dont know whether the fragment belongs to either H1 or H2, 